# METATUTU Demo Set #1

This demo set is including the general tools.

## 👉 Search Path

If you want to invoke the METATUTU library with source code, please run below code **FIRST** to setup the correct search path.
Otherwise, simply skip them.  It's strongly suggest to not use source code directly.  Instead, as library developer, if you want to run demo with latest source code, you may make an installation with code with given build tool.

In [ ]:
# import sys
# import os
# METATUTU_PATH = os.path.abspath("../../lib")
# if METATUTU_PATH not in sys.path: sys.path.insert(0, METATUTU_PATH)

## 📘 metatutu.debugging

### 📄 Clocker

In [ ]:
import sys
import time
from metatutu.debugging import Clocker

# test 1
clocker = Clocker()
time.sleep(1.2)
clocker.record("p1")

time.sleep(0.5)
clocker.record("p2")

print(clocker.results_text())

# test 2
with clocker:
	time.sleep(0.3)
print(clocker.results_text())

## 📘 metatutu.logging

### 📄 Multiple output logging

In [ ]:
import random
from metatutu.logging import *

# initialize console logger
console_logger = ConsoleLogger()
console_logger.include_timestamp = False

# initialize file logger
file_logger = FileLogger(r"d:\data\test_project\test.log")
file_logger.async_mode = True
file_logger.auto_flush = True

# initialize dated file logger
dated_file_logger = DatedFileLogger(r"d:\data\test_project\logtest")

# initialize aggregator of loggers
logger = Loggers()
logger.register_logger(console_logger)
logger.register_logger(file_logger)
logger.register_logger(dated_file_logger)

# log test data
logger.separator()
logger.log("This is output of running example code:")
for task_id in range(0, 100):
	logger.info("Start task {0}:".format(task_id + 1), 1)
	
	task_desc = "Task ID: {0}\r\n".format(task_id + 1)
	task_desc += "Task Desc: Do something..."
	logger.log(task_desc, 2)

	try:
		result = random.randint(0, 100)
		if result < 20:
			logger.log("General information is being logged.", 2)
		elif result < 40:
			logger.debug("Debug information is being logged.", 2)
		elif result < 50:
			logger.info("Some program information is being logged.", 2)
		elif result < 60:
			logger.warning("Warning information is being logged.", 2)
		elif result < 80:
			logger.error("Error is being logged.", 2)
		elif result < 90:
			logger.critical("Critical error is being logged.", 2)
		else:
			temp = 1 / 0
	except Exception as ex:
		logger.exception(ex, depth=2)

	logger.info("End task {0}:".format(task_id + 1), 1)

# close loggers
logger.close()

## 📘 metatutu.fsds

### 📄 Project workspace

In [ ]:
import datetime
from metatutu.fsds import *

def test_fsds_project_workspace():
	fsds = FileSystemDataStore(r"d:\data\test_project")

	print(fsds.get_path("common.csv"))
	print(fsds.get_path(r"release\version.txt"))
	print(fsds.get_dated_path("source.csv"))
	print(fsds.get_dated_path("source.csv", "2022-03-11"))
	print(fsds.get_dated_path("source.csv", datetime.datetime(2021, 12, 31)))

test_fsds_project_workspace()

### 📄 List files

In [ ]:
from metatutu.fsds import *

def test_fsds_list_files():
    fsds = FileSystemDataStore(r"d:\data\test_project")

    folderpath = fsds.get_parent_path("any")  # root path
    print(folderpath)
    print(fsds.list_files(folderpath))
    print(fsds.list_files(folderpath, filters=[".jpg"]))
    
test_fsds_list_files()

### 📄 Text file I/O & Temp file/folder

In [ ]:
import datetime
import os
import time
from metatutu.fsds import *

def test_fsds_text_file():
	fsds = FileSystemDataStore(r"d:\data\test_project")

	tmp = fsds.create_temp_file()
	filepath = tmp.path
	fsds.save_file_contents(filepath, "Hello World!")
	contents = fsds.load_file_contents(filepath)
	if contents:
		print(contents)
	else:
		print("Failed!")

	tmp2 = fsds.create_temp_folder()
	filepath = os.path.join(tmp2.path, "greetings.txt")
	fsds.save_file_contents(filepath, "How are you?")
	contents = fsds.load_file_contents(filepath)
	if contents:
		print(contents)
	else:
		print("Failed!")

	time.sleep(10)  # give chance to check file system before temp file/folder deleted

test_fsds_text_file()

## 📘 metatutu.images

### 📄 Stitch images

In [ ]:
from PIL import Image
from metatutu.fsds import *
from metatutu.images import *

def test_images_stitch_images():
	fsds = FileSystemDataStore(r"d:\data\test_project")

	filepaths = []
	for i in range(0, 5):
		filepath = fsds.get_path("{}.png".format(i + 1))
		filepaths.append(filepath)
		image = Image.new("RGBA", (640, 480), (i * 30, i * 30, i * 30, 255))
		image.save(filepath)

	images = Images()
	images.append_files(filepaths)

	filepath = fsds.get_path("stitched-y.png")
	images.stitch(True).save(filepath)
	print(filepath)

	filepath = fsds.get_path("stitched-x.png")
	images.stitch(False).save(filepath)
	print(filepath)
    
test_images_stitch_images()

### 📄 Create GIF

In [ ]:
from metatutu.fsds import *
from metatutu.images import *

def test_images_create_gif():
	fsds = FileSystemDataStore(r"d:\data\test_project")

	filepaths = []
	for i in range(0, 5):
		filepaths.append(fsds.get_path("{}.png".format(i + 1)))

	images = Images()
	images.append_files(filepaths)

	filepath = fsds.get_path("test.gif")
	images.create_gif(filepath, 0.5)
	print(filepath)
    
test_images_create_gif()

## 📘 metatutu.pipeline

### 📄 Individual

In [ ]:
import time
from metatutu.pipeline import *
from metatutu.debugging import Clocker

class TestDoer(Doer):
	def __init__(self, maxsize):
		Doer.__init__(self, maxsize)

	def _start_working(self):
		print("doer: start working")

	def _stop_working(self):
		print("doer: stop working")

	def _process_task(self, task):
		print("doer ({0}): start task {1}".format(self.id, task["id"]))
		time.sleep(0.2)	# simulate the length of process time
		print("doer ({0}): end task {1}".format(self.id, task["id"]))

clocker = Clocker()

# hire doer
clocker.reset()
doer = TestDoer(0)
doer.hire()
clocker.record("hire")

# feed tasks
for i in range(0, 10):
	doer.task_queue.push_task({"id": str(i + 1)})
doer.task_queue.push_task({"id": "p50"}, 50)
doer.task_queue.push_task({"id": "p10"}, 10)
doer.task_queue.push_task({"id": "p20"}, 20)
doer.task_queue.push_task({"id": "p200"}, 200)
clocker.record("push")

# wait and dismiss doer
doer.dismiss()
clocker.record("wait & dismiss")

# show result
print(clocker.results_text())

### 📄 Team

In [ ]:
import random
from metatutu.pipeline import *
from metatutu.debugging import *

class TestOperator(Operator):
	def __init__(self):
		Operator.__init__(self)
		self.team = None
		self.task_queue = None
		self.logger = None

	def _start_working(self):
		print("operator ({0}): start working".format(self.id))

	def _stop_working(self):
		print("operator ({0}): stop working".format(self.id))

	def _process_task(self, task):
		print("operator ({0}): start task {1}".format(self.id, task["id"]))
		time.sleep(random.randint(1, 5) / 10)	# simulate the length of process time
		print("operator ({0}): end task {1}".format(self.id, task["id"]))

	def _pop_task(self):
		if self.task_queue is None: return None
		return self.task_queue.pop_task()

	def bind(self, data):
		self.team = data
		self.task_queue = data.task_queue

class TestManager(Controller):
	def __init__(self):
		Controller.__init__(self)
		self.team = None

	def _process(self):
		while True:
			# check stop request
			if self._dismissNotice.is_set(): break

			# get task queue status
			status = self.team.task_queue.get_status()
			queue_count = status["count"]
			msg = "task queue\r\n"
			msg += "  total count: {0}\r\n".format(status["total_count"])
			msg += "  peak count: {0}\r\n".format(status["peak_count"])
			msg += "  count: {0}\r\n".format(queue_count)

			# get workers status
			status = self.team.operators.get_status()
			operator_count = status["count"]
			operator_idle_count = status["idle_count"]
			operator_idle_rate = status["idle_rate"]
			msg += "operators\r\n"
			msg += "  total count: {0}\r\n".format(status["total_count"])
			msg += "  peak count: {0}\r\n".format(status["peak_count"])
			msg += "  count: {0}\r\n".format(operator_count)
			msg += "  idle count: {0}\r\n".format(operator_idle_count)
			msg += "  idle rate: {0:.1f}%".format(operator_idle_rate * 100)
			print(msg)

			# adjust operators
			if operator_idle_rate > 0.8:
				if operator_idle_count > queue_count and operator_count > 1:
					self.team.dismiss_operator(1)
					print("manager: dismissed an operator")
			if operator_idle_rate < 0.2:
				if operator_idle_count < queue_count and operator_count < 50: 
					self.team.hire_operator(1)
					print("manager: hired an operator")

			# sleep
			time.sleep(0.1)

class TestTeam(Team):
	def __init__(self):
		Team.__init__(self)
		self.operator_class = TestOperator
		self.manager = None

	def __del__(self):
		Team.__del__(self)
	
	def hire(self):
		# create task queue
		self.task_queue = TaskQueue(0)

		# hire manager
		self.manager = TestManager()
		self.manager.team = self
		self.manager.hire()

		# hire initial operators
		self.hire_operator(1)

	def dismiss(self):
		# finish all tasks
		self.finish_all_tasks()

		# dismiss operators
		self.operators.dismiss_all()

		# dismiss manager
		self.manager.dismiss()
		del self.manager
		self.manager = None

clocker = Clocker()

# hire team
clocker.reset()
team = TestTeam()
team.hire()
clocker.record("hire")

# feed tasks
for i in range(0, 10):
	team.task_queue.push_task({"id": str(i + 1)})
team.task_queue.push_task({"id": "p50"}, 50)
team.task_queue.push_task({"id": "p10"}, 10)
team.task_queue.push_task({"id": "p20"}, 20)
team.task_queue.push_task({"id": "p200"}, 200)
clocker.record("push")

# wait and dismiss team
team.dismiss()
clocker.record("wait & dismiss")

# show result
print(clocker.results_text())